# eBay Car Sales #

This is project to explore ebay data and find the main contributing factor in the car price.  Is it car mileage or car brand?

In this project the following would need to be done:

* Clean Data (e.g. many numerical fields stored as strings, columns convert from Camel to Snake case etc)
* Analyze car listings


In [1]:
import pandas as pd
import numpy as np

autos = pd.read_csv('autos.csv', encoding='Latin-1')

In [2]:
autos.info()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 20 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   dateCrawled          50000 non-null  object
 1   name                 50000 non-null  object
 2   seller               50000 non-null  object
 3   offerType            50000 non-null  object
 4   price                50000 non-null  object
 5   abtest               50000 non-null  object
 6   vehicleType          44905 non-null  object
 7   yearOfRegistration   50000 non-null  int64 
 8   gearbox              47320 non-null  object
 9   powerPS              50000 non-null  int64 
 10  model                47242 non-null  object
 11  odometer             50000 non-null  object
 12  monthOfRegistration  50000 non-null  int64 
 13  fuelType             45518 non-null  object
 14  brand                50000 non-null  object
 15  notRepairedDamage    40171 non-null  object
 16  date

There are 20 columns with 5 integer values.
Some of the columns have null values but generally less then 20% per column
Column names are writtin in inconsistent Camel case with some of the names using 0 instead of O.

In [3]:
autos.describe()

,yearOfRegistration,powerPS,monthOfRegistration,nrOfPictures,postalCode
count,50000.000000,50000.000000,50000.000000,50000.0,50000.000000
mean,2005.073280,116.355920,5.723360,0.0,50813.627300
std,105.712813,209.216627,3.711984,0.0,25779.747957
min,1000.000000,0.000000,0.000000,0.0,1067.000000
25%,1999.000000,70.000000,3.000000,0.0,30451.000000
50%,2003.000000,105.000000,6.000000,0.0,49577.000000
75%,2008.000000,150.000000,9.000000,0.0,71540.000000
max,9999.000000,17700.000000,12.000000,0.0,99998.000000


There are 50,000 rows with mean falling on year 2005.  
Minimum appears to be incorrect year 1000 and maximum 9,999 - some cleaning to be done of unrealistic numbers. 
Postal_code needs further investigation

In [4]:
autos.columns

Index(['dateCrawled', 'name', 'seller', 'offerType', 'price', 'abtest',
       'vehicleType', 'yearOfRegistration', 'gearbox', 'powerPS', 'model',
       'odometer', 'monthOfRegistration', 'fuelType', 'brand',
       'notRepairedDamage', 'dateCreated', 'nrOfPictures', 'postalCode',
       'lastSeen'],
      dtype='object')

In [5]:
def camel_to_snake(str):
    str = convert_exception(str)
    converted = [str[0].lower()]
    for char in str[1:]:
        if char in ('ABCDEFGHIJKLMNOPQRSTUVWXYZ'):
            converted.append('_')
            converted.append(char.lower())
        else:
            converted.append(char)
    return ''.join(converted)

def convert_exception(str):
    if str == 'yearOfRegistration':
        str = 'registration_year'
        return str
    elif str == 'monthOfRegistration':
        str = 'registration_month'
        return str
    elif str == 'notRepairedDamage':
        str = 'unrepaired_damage'
        return str
    elif str == 'dateCreated':
        str = 'ad_created'
        return str
    else:
        return str
    

new_cols = []
for col in autos.columns:
    new_col = camel_to_snake(col)
    new_cols.append(new_col)

autos.columns = new_cols

There are two objectives  in the step above converting from camel to snake case:
1. deal with exceptions to naming
2. convert to snake case

I created 2 functions to deal with both of those and return column transformed to requirements.

Now I needed to loop through the existing columns creating a list with new names.  

Once the list completed simply assign the new list back to the columns attributes to rename.

In [6]:
autos.head()

,date_crawled,name,seller,offer_type,price,abtest,vehicle_type,registration_year,gearbox,power_p_s,model,odometer,registration_month,fuel_type,brand,unrepaired_damage,ad_created,nr_of_pictures,postal_code,last_seen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01 00:00:00,0,39218,2016-04-01 14:38:50


In [7]:
autos.describe(include='all')

,date_crawled,name,seller,offer_type,price,abtest,vehicle_type,registration_year,gearbox,power_p_s,model,odometer,registration_month,fuel_type,brand,unrepaired_damage,ad_created,nr_of_pictures,postal_code,last_seen
count,50000,50000,50000,50000,50000,50000,44905,50000.000000,47320,50000.000000,47242,50000,50000.000000,45518,50000,40171,50000,50000.0,50000.000000,50000
unique,48213,38754,2,2,2357,2,8,NaN,2,NaN,245,13,NaN,7,40,2,76,NaN,NaN,39481
top,2016-03-23 19:38:20,Ford_Fiesta,privat,Angebot,$0,test,limousine,NaN,manuell,NaN,golf,"150,000km",NaN,benzin,volkswagen,nein,2016-04-03 00:00:00,NaN,NaN,2016-04-07 06:17:27
freq,3,78,49999,49999,1421,25756,12859,NaN,36993,NaN,4024,32424,NaN,30107,10687,35232,1946,NaN,NaN,8
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2005.073280,NaN,116.355920,NaN,NaN,5.723360,NaN,NaN,NaN,NaN,0.0,50813.627300,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,105.712813,NaN,209.216627,NaN,NaN,3.711984,NaN,NaN,NaN,NaN,0.0,25779.747957,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1000.000000,NaN,0.000000,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,0.0,1067.000000,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1999.000000,NaN,70.000000,NaN,NaN,3.000000,NaN,NaN,NaN,NaN,0.0,30451.000000,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2003.000000,NaN,105.000000,NaN,NaN,6.000000,NaN,NaN,NaN,NaN,0.0,49577.000000,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2008.000000,NaN,150.000000,NaN,NaN,9.000000,NaN,NaN,NaN,NaN,0.0,71540.000000,NaN


The following columns might need to be dropped or NaN rows eliminated:
* date_crawled: drop NaN rows
* name: drop NaN rows & extract car make 
* offer_type 
* seller 
* abtest columns has 2 values - how useful is it? - drop
* registration_month - remove NaN rows

The following columns have numeric data stored as text and need conversion:
* price 
* odometer 
* ad_created
* last_seen

The following columns require further investigation and action:
* registration year - eliminate incorrect year data or substitute with data on the basis of assumption 
* power_p_s
* model & name - determine what data is actually useful and cleanse


In [8]:
autos['odometer'] = (autos['odometer']
      .str.replace('km','')
      .str.replace(',','')
      .str.strip()
      .astype(int)
    )

autos['price'] = (autos['price']
      .str.replace('$','')
      .str.replace(',','')
      .str.strip()
      .astype(float)
    )



In [9]:
autos.rename(columns={'odometer':'odometer_km'}, inplace=True)

In [10]:
print(autos['odometer_km'].unique().shape)
print()
print()
print(autos['odometer_km'].describe())
print()
print()
print(autos['odometer_km'].value_counts().sort_index(ascending=False).head())

(13,)


count     50000.000000
mean     125732.700000
std       40042.211706
min        5000.000000
25%      125000.000000
50%      150000.000000
75%      150000.000000
max      150000.000000
Name: odometer_km, dtype: float64


150000    32424
125000     5170
100000     2169
90000      1757
80000      1436
Name: odometer_km, dtype: int64


In [11]:
print(autos['price'].unique().shape)
print()
print()
print(autos['price'].describe())
print()
print()
print(autos['price'].value_counts().sort_index(ascending=False).head(10))
print()
print()
print(autos['price'].value_counts().sort_index(ascending=True).head(10))

(2357,)


count    5.000000e+04
mean     9.840044e+03
std      4.811044e+05
min      0.000000e+00
25%      1.100000e+03
50%      2.950000e+03
75%      7.200000e+03
max      1.000000e+08
Name: price, dtype: float64


99999999.0    1
27322222.0    1
12345678.0    3
11111111.0    2
10000000.0    1
3890000.0     1
1300000.0     1
1234566.0     1
999999.0      2
999990.0      1
Name: price, dtype: int64


0.0     1421
1.0      156
2.0        3
3.0        1
5.0        2
8.0        1
9.0        1
10.0       7
11.0       2
12.0       3
Name: price, dtype: int64


Cars with the price over 150,000 can be removed (unless we are targetting luxury segment) particularly as some of the data is missing or inconsistent

Similarly remove cars with price less then $0 as they are not a transaction

In [12]:
autos.drop(autos.index[autos['price'] == 0].tolist(), inplace=True)

In [13]:
autos.drop(autos.index[autos['price'] > 150000].tolist(),inplace=True)

In [14]:
autos[['date_crawled','ad_created','last_seen']][0:5]

,date_crawled,ad_created,last_seen
0,2016-03-26 17:47:46,2016-03-26 00:00:00,2016-04-06 06:45:54
1,2016-04-04 13:38:56,2016-04-04 00:00:00,2016-04-06 14:45:08
2,2016-03-26 18:57:24,2016-03-26 00:00:00,2016-04-06 20:15:37
3,2016-03-12 16:58:10,2016-03-12 00:00:00,2016-03-15 03:16:28
4,2016-04-01 14:38:50,2016-04-01 00:00:00,2016-04-01 14:38:50


In [15]:
the_date_crawled = (autos['date_crawled'].str[:10].value_counts(normalize=True, dropna=False)*100).sort_index()

the_date_crawled.describe()

count    34.000000
mean      2.941176
std       0.976483
min       0.140076
25%       2.996704
50%       3.277372
75%       3.482336
max       3.860336
Name: date_crawled, dtype: float64

In [16]:
print(the_date_crawled.describe())

from datetime import datetime
d1 = datetime.strptime('2016-03-05', "%Y-%m-%d")
d2 = datetime.strptime('2016-04-07', "%Y-%m-%d")

print((d2 - d1).days)

count    34.000000
mean      2.941176
std       0.976483
min       0.140076
25%       2.996704
50%       3.277372
75%       3.482336
max       3.860336
Name: date_crawled, dtype: float64
33


DATE CRAWLED

Generally crawls are distributed uniformely every day.  The crawl occurred between 2016-Mar-05 and 2016-Apr-07

* therea are 34 distinct dates
* data mostly from March 16 and there is no missin dates
* the lowest crawl appears to be on Sundays
* there seems to be no crawling before March 16 even though the ads have been created from Jun 15

In [17]:
ad_created = (autos['ad_created'].str[:10].value_counts(normalize=True, dropna=False)*100).sort_index()
ad_created.describe()

count    76.000000
mean      1.315789
std       1.595677
min       0.002060
25%       0.002060
50%       0.014420
75%       3.210938
max       3.885055
Name: ad_created, dtype: float64

In [18]:
print(ad_created.describe())

from datetime import datetime
d1 = datetime.strptime('2015-06-11', "%Y-%m-%d")
d2 = datetime.strptime('2016-04-07', "%Y-%m-%d")

print((d2 - d1).days)

count    76.000000
mean      1.315789
std       1.595677
min       0.002060
25%       0.002060
50%       0.014420
75%       3.210938
max       3.885055
Name: ad_created, dtype: float64
301


AD CREATED
* there are 76 distinct values
* the ad creation spanned between 2015-Jun-11 and 2016-Apr-07.  
* interesting that there has been no crawls for several months.

In [19]:
last_seen = (autos['last_seen'].str[:10].value_counts(normalize=True, dropna=False)*100).sort_index()
last_seen.describe()

count    34.000000
mean      2.941176
std       4.372103
min       0.107117
25%       1.243691
50%       1.949737
75%       2.379236
max      22.185601
Name: last_seen, dtype: float64

LAST SEEN

* Aligns with the crawling dates and has 34 distinct dates over mostly March 16
* Did not crawl earlier

In [20]:
print(autos['registration_year'].describe())
print()
print()
print(autos['registration_year'].value_counts(normalize=True,dropna=False).sort_index().head())
print()
print()
print(autos['registration_year'].value_counts(normalize=True,dropna=False).sort_index().tail())



count    48545.000000
mean      2004.753445
std         88.661893
min       1000.000000
25%       1999.000000
50%       2004.000000
75%       2008.000000
max       9999.000000
Name: registration_year, dtype: float64


1000    0.000021
1001    0.000021
1111    0.000021
1800    0.000041
1910    0.000103
Name: registration_year, dtype: float64


5911    0.000021
6200    0.000021
8888    0.000021
9000    0.000021
9999    0.000062
Name: registration_year, dtype: float64


In [21]:
print(autos.loc[autos['registration_year'] <= 1910,['name','model','registration_year','odometer_km','ad_created']])

                                        name    model  registration_year  \
3679                              Suche_Auto      NaN               1910   
10556                             UNFAL_Auto      NaN               1800   
22316  VW_Kaefer.__Zwei_zum_Preis_von_einem.   kaefer               1000   
22659                           Opel_Corsa_B    corsa               1910   
24511            Trabant__wartburg__Ostalgie      NaN               1111   
28693                         Renault_Twingo      NaN               1910   
30781       Opel_Calibra_V6_DTM_Bausatz_1:24  calibra               1910   
32585                             UNFAL_Auto      NaN               1800   
45157                             Motorhaube      NaN               1910   
49283                             Citroen_HY   andere               1001   

       odometer_km           ad_created  
3679          5000  2016-04-04 00:00:00  
10556         5000  2016-04-01 00:00:00  
22316         5000  2016-03-29 00:00:

REGISTRATION YEAR

There are clearly following outlier years:
* 1800 - there were no cars
* Spot check of the cars in early 1900 shows that the date is relates to modern models that couldn't have been registered before they were manufactured.
* 2017 onwards - since the ads have been last seen in Apr 2016.
* Anything inclusive: 2017 - 9999 or 1000 - 1910. There are 1981 cars that fall into that category and need be removed


In [22]:
#Remove all cars before 1910 and after 2017

autos = autos[(autos['registration_year'] > 1910) & (autos['registration_year'] < 2017)]

In [23]:
#print(autos['registration_year'][(autos['registration_year'] <= 1910) | (autos['registration_year'] >= 2017)].count())

autos['registration_year'].value_counts() #+1981

autos['registration_year'][autos['registration_year'] >= 1994].count()/autos['registration_year'].count()

0.9405864197530864

Absolute majority of the cars 45,049 have been registered from 1994 onward (inclusive).  This is 94% of the cars


In [133]:
print(autos['brand'].describe())
print()
print()
#print(autos['brand'].value_counts(normalize=True, dropna=False, sort=True).head(20)) # sorting by frequency
idx = autos['brand'].value_counts(normalize=True, dropna=False, sort=True).head(10).index
print()
print('----')
print()
#print(autos['brand'][['price']])
#pd.options.display.float_format = '${:,.0f}'.format
#pd.reset_option('^display.', silent=True)
grp_price = autos.groupby('brand')['price'].mean()
grp_price = grp_price[list(idx)]

# print(autos.loc[autos['brand'].isin(idx),['brand','vehicle_type','price']].mean())
print(autos.loc[:,'price'].mean())
print()
print('----')
print()
print(grp_price)

count          46656
unique            40
top       volkswagen
freq            9862
Name: brand, dtype: object



----

5886.792116769548

----

brand
volkswagen       5402.410262
bmw              8201.538566
opel             2976.321713
mercedes_benz    8554.524106
audi             9295.681683
ford             3749.469507
renault          2475.717273
peugeot          3094.017229
fiat             2813.748538
seat             4397.230950
Name: price, dtype: float64


Selecting top 10 cars for further investigation.  Observation - 7 top brands by sale transaction constitute 70% of all brands on sale on eBay during the period. 


In [135]:
idx = autos['brand'].value_counts(normalize=True, dropna=False, sort=True).head(10).index
dict_brand_price = {}

for i in idx:
    a_mean_price = autos.loc[autos['brand']== i,'price'].mean()
    dict_brand_price[i] = a_mean_price

#the_values = sorted(dict_brand_price.values(), reverse=True)
    
for k in dict_brand_price:
    print(k, dict_brand_price[k])



volkswagen 5402.410261610221
bmw 8201.538566419946
opel 2976.3217131474103
mercedes_benz 8554.524105754277
audi 9295.681683168317
ford 3749.4695065890287
renault 2475.7172727272728
peugeot 3094.0172290021537
fiat 2813.748538011696
seat 4397.230949589683


Some discarded ways to sort:


sorted_brands_by_price = sorted(dict_brand_price.items(), key=lambda x: x[1], reverse=True)
for k in dict_brand_price:
    print(k, ' : ', dict_brand_price[k])
[print(k,v) for (k,v) in sorted(dict_brand_price.items(), key=lambda x: x[1])]
import operator
sorted_dict = sort(dict_brand_price.items(), key=operator.itemgetter(1), reverse=True)

Mean price of all cars sold on eBay - $5,886.8.
The most expensive brands mercedes, audi and bmw command on average premium of 25 to 36 percent.  
Volkswagen is positioned right in the region of mean price with bottom priced sector occupied by Renault, Fiat and Opel


In [138]:
idx = autos['brand'].value_counts(normalize=True, dropna=False, sort=True).head(10).index
dict_brand_odo = {}

for i in idx:
    a_mean_odo = autos.loc[autos['brand']== i,'odometer_km'].mean()
    dict_brand_odo[i] = a_mean_odo

for k in dict_brand_odo:
    print(k, dict_brand_odo[k])

volkswagen 128707.15879132022
bmw 132633.4242306194
opel 129311.75298804781
mercedes_benz 130826.48300377694
audi 129188.11881188118
ford 124266.01287159056
renault 128127.27272727272
peugeot 127153.62526920316
fiat 117121.9715956558
seat 121131.30128956624


In [139]:
bmprice_series = pd.Series(dict_brand_price)
bmodo_series = pd.Series(dict_brand_odo)

In [144]:
df_bmprice = pd.DataFrame(bmprice_series,columns=['mean_price'])
df_bmodo = pd.DataFrame(bmodo_series, columns=['mean_odo'])

In [147]:
pd.options.display.float_format = '${:,.0f}'.format
print(df_bmprice)
pd.options.display.float_format = '{:,.0f}'.format
print(df_bmodo)
pd.reset_option('^display.', silent=True)

               mean_price
volkswagen         $5,402
bmw                $8,202
opel               $2,976
mercedes_benz      $8,555
audi               $9,296
ford               $3,749
renault            $2,476
peugeot            $3,094
fiat               $2,814
seat               $4,397
               mean_odo
volkswagen      128,707
bmw             132,633
opel            129,312
mercedes_benz   130,826
audi            129,188
ford            124,266
renault         128,127
peugeot         127,154
fiat            117,122
seat            121,131


There is no relationship between mean price and mileage.  All cars have on average the same milage circa 120-130K with prices significantly differing by brand.  

There is a strong correlation between the brand and how well it holds price despite mileage.  This can be because of the perception of status or actual difference in quality or combination of both.  